In [ ]:
# import llm as L
# L.TOKEN_COUNT_PATH = '/data/ai_club/team_3_2024-25/tokcounts2/'

In [ ]:
import json

In [150]:
allowed_vocab = [ # can collapse
    # greetings
    'terve', 'hei',

    # nouns
    'talo',     # house
    'vesi',     # water
    'ystävä',   # friend
    'huomenta', # morning
    'velho',    # wizard
    'suomi',    # Finland
    'koira',    # dog
    'nimi',     # name

    # singular possessive nouns for 'nimi'
    'nimeni',   # first person "my name"
    'nimesi',   # second person "your name"
    'nimensä',  # third person "his name"

    # singular posessive nous for 'ystävä'
    'ystäväni',  # first person
    'ystäväsi',  # second person
    'ystävänsä', # third person

    # adjectives
    'vanha',       # old
    'hyvää',       # good
    'suomalainen', # Finnish
    'mukava',      # nice

    # pronouns, posesives, "to be" verbs
    'minä', 'minun', 'olen', 'olenko', # first person
    'sinä', 'sinun', 'olet', 'oletko', # second person
    'hän', 'hänen', 'on', 'onko',      # third person

    # names
    'matti', 'aleksi', 'sami'

    # useful words
    'kyllä', # yes
    'ei', # no
    'mitä', # "what/how" as in "what did you say?" or "how are you" -- about more abstract things
    'mikä', # "what" as in "what is this?" or "what is your name?" -- about specific things
]

In [1]:
from openai import OpenAI

client = OpenAI(
    base_url = "http://dh-dgxh100-2.hpc.msoe.edu:8000/v1",
    api_key = "not_used"
)

---
### Approach: restart whole sentence w/ feedback.

In [195]:
messages = [
    { "role": "system", "content":
        'You are an Finnish language teacher. ' +
        'Give short responses, ideally one sentence. Use lots of emojis.\n\n' +
        'This is the set of allowed vocab you can draw from for responses:\n{' + ', '.join(allowed_vocab) + '}' +
        '\nIMPORTANT: All word usage must be grammatically correct Finnish'
    },
    {
        "role": "user",
        "content": 'You start. Keep this interactive by asking ME questions.'
    },
    {
        "role": "assistant",
        "content": 'Hei 🤗, mitä sinun nimi on? 🤔'
    },
    {
        "role": "user",
        "content": 'Nimeni on Lucas'
    },
    {
        "role": "assistant",
        "content": 'Mukava 🤝, oletko sinä suomalainen 🇫🇮?'
    },
    {
        "role": "user",
        "content": 'En ole. Olen amerikkalainen'
    },
]



out = client.chat.completions.create(
    model="meta/llama-3.3-70b-instruct",
    messages=messages,
    max_tokens=1024,
    stream=True,
    temperature=0,
    # response_format={'type': 'json_object'}
)

# for t in out:
#     tok = t.choices[0].delta.content
#     print(tok, end='')

current_word = ''
for t in out:
    t = t.choices[0].delta.content
    t = ''.join(c for c in t if c.isalpha() or c.isspace())

    # print(t)

    has_space = ' ' in t
    t = t.strip()

    # print(f"'{t}' ({has_space})")

    if has_space:
        print(current_word)
        current_word = ''
        
    current_word += t


Hyvää

minä
olen
Aleksi
ja
opetan
suomea

mikä
on
sinun
suosikki
eläin
koira
vai
joku
muu


---
### Approach in the middle of ^ and V: go back a few words with every mistake

In [ ]:
# will do l8r

---
### Approach: correct individual words

In [170]:
' '.join(output)

'hei, mitä sinä olet? mukava koira? sinun nimi on matti ei, aleksi sinä'

In [167]:
output = []
mistakes = []

while True:
    output_str = ' '.join(output)
    mistakes_str = (
        '\n\nRemember, you CANNOT use these words: ' + ', '.join(mistakes)
        if mistakes else
        ''
    )

    messages = [
        { "role": "system", "content":
            'You are an Finnish language teacher. ' +
            'Give short responses, ideally one sentence. Use lots of emojis (they count as words here).\n\n' +
            'This is the set of allowed vocab you can draw from for responses:\n{' + ', '.join(allowed_vocab) + '}' +
            '\nIMPORTANT: All word usage must be grammatically correct Finnish\n\n' +
            'Your responses must continue given ones. Your continuations can be: a word (optionally with a punctuation symbol).'
        },
        {
            "role": "user",
            "content": f'You start. Keep this interactive by asking ME questions.\n\nYour response should continue:\n{output_str}' +
            mistakes_str +
            '\n\nRespond in this json and nothing else: {"next_word": "...", "end?": "yes/no"}',
        }
    ]

    out = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",
        messages=messages,
        max_tokens=100,
        # stream=True,
        temperature=0,
        # response_format={'type': 'json_object'}
    )

    # for t in out:
    #     tok = t.choices[0].delta.content
    #     print(tok, end='')

    out = json.loads(out.choices[0].message.content)

    # print(out)

    word_raw = out['next_word'].split()[0].lower()
    word = ''.join([c for c in word_raw if c.isalnum()])

    if word in allowed_vocab:
        print(word_raw, end=' - ')
        output.append(word_raw)
    else:
        print('X', word_raw, end=' - ')
        mistakes.append(word)

    print(out['end?'])

    if out['end?'].lower() == 'yes':
        break

hei, - no
mitä - no
sinä - no
X teet? - no
olet? - no
mukava - no
koira? - no
sinun - no
nimi - no
on - no
matti - no
ei, - no
aleksi - no
sinä - no


KeyboardInterrupt: 